In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
%config Completer.use_jedi = False
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
test_Data=pd.read_csv('../input/spaceship-titanic/test.csv')
train_Data=pd.read_csv('../input/spaceship-titanic/train.csv')

train_Data.info()
train_Data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [3]:
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder, Binarizer ,LabelEncoder
from tensorflow import unique

def pp_Data(data,train=False):
    data.info()
    bool_var=list()
    cat_var=list()
    drop_var=list()
    lenc=LabelEncoder()
    
    for x in range(0,len(data.columns)):
        col=data.columns[x]
        if col== 'PassengerId':
            continue
            
        if data.dtypes[x]=='bool':
            bool_var.append(col)
        elif data.dtypes[x]=='object':
            lenc.fit(data[col])
            uc=lenc.classes_
            if len(uc) == 2 :
                bool_var.append(col)
                continue
            if len(uc) < len(data)/10:
                cat_var.append(col)
            else :
                drop_var.append(col)
    for x in data.columns:
        if(data[x].isnull().any()):
            if x in cat_var or x in bool_var:
                print("has null",x)
                data[x].fillna(data[x].mode()[0],inplace=True)
                #print(data[x].mode()[0])
            else:
                
                data[x].fillna(data[x].mode()[0],inplace=True)
        
    data.info()
    
    data.drop(drop_var,axis=1,inplace=True)
    
    bin=Binarizer()
    oenc=OneHotEncoder()
    pp_bool=pd.DataFrame()
    pp_cat=pd.DataFrame()
    print(bool_var,cat_var,data.columns)
    
    for x in bool_var:
        temp= pd.DataFrame(lenc.fit_transform( pd.DataFrame(data[x]) ))
        #print(lenc.classes_)
        temp.columns=[x]
        #print(temp.head())
        
        if len(pp_bool)==0:
            pp_bool=temp
        else:    
            pp_bool= pd.concat([pp_bool,temp],axis=1)
        
    #print(pp_bool.head())
    
    for x in cat_var:
        #print(x)
        temp= pd.DataFrame(oenc.fit_transform( pd.DataFrame(data[x]) ).toarray()) 
        #print(oenc.categories_)
        temp.columns=oenc.categories_
        
        if len(pp_cat)==0:
            pp_cat = temp
        else:    
            pp_cat = pd.concat([pp_cat,temp ],axis=1)
    #pp_cat.info()    
    #pp_bool.info()
    
    #col.append('Survived')
    test_y=pd.DataFrame
    if not train:
        test_y= pd.DataFrame(data['Transported'])
    #print(test_y.head())
    col=bool_var+cat_var
    #print(col)
    #print(len(pp_cat),len(pp_bool),len(data))
    test_x=data.drop(col,axis=1);
    test_x.join(pp_bool)
    test_x.join(pp_cat)
    #print(test_x.columns)
    #pd.([pp_cat,pp_bool,data.drop(col,axis=1)])
    #print("here",len(test_x))
    return test_x,test_y
    
    
    
test_x,test_y= pp_Data(train_Data)
print(test_x.describe())
print(test_y.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB
has null HomePlanet
has null CryoSleep
has null Destination
has null VIP
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 

/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [4]:
#cor=test_x.corr()
#cov=test_x.cov()
#cor.plot.bar(y='Transported')
#cov.plot.bar(y='Transported')



In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import HalvingGridSearchCV


print("here=>",len(test_x),len(test_y))

#test_x.drop('Transported',inplace=True,axis=1)

train_X, test_X, train_Y, test_Y= train_test_split(test_x,test_y,random_state=26,test_size=0.2)
train_X.info()
train_Y.info()
model = RandomForestClassifier(random_state=21,max_depth=5, verbose=3)
model.fit(train_X,train_Y)
pred=model.predict(test_X)
error=mean_absolute_error(test_Y,pred)
print(error)
#print(model.get_params())
#params={"max_depth":[5,20],"min_samples_split":[2,10],"verbose":[0,3]}
#search= HalvingGridSearchCV(model,params).fit(train_X,train_Y)
#print(search.best_params_)
#pred=search.predict(test_X)




here=> 8693 8693
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6954 entries, 7028 to 4917
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   6954 non-null   object 
 1   Age           6954 non-null   float64
 2   RoomService   6954 non-null   float64
 3   FoodCourt     6954 non-null   float64
 4   ShoppingMall  6954 non-null   float64
 5   Spa           6954 non-null   float64
 6   VRDeck        6954 non-null   float64
dtypes: float64(6), object(1)
memory usage: 434.6+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6954 entries, 7028 to 4917
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Transported  6954 non-null   bool 
dtypes: bool(1)
memory usage: 61.1 KB
building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
build

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [6]:
TEST_X,TEST_Y= pp_Data(test_Data,train=True)
pred=pd.DataFrame(model.predict(TEST_X))
pred.columns=['Transported']
submission=pd.DataFrame(test_Data['PassengerId'])
submission=submission.join(pred)
submission.to_csv('submission.csv', index=False)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB
has null HomePlanet
has null CryoSleep
has null Destination
has null VIP
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column   

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
